In [18]:
# -*- coding: utf-8 -*-

import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', 15)

df = sns.load_dataset('titanic')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [19]:
rdf = df.drop(['deck', 'embark_town'], axis=1)
print(rdf.columns.values)

['survived' 'pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked' 'class'
 'who' 'adult_male' 'alive' 'alone']


In [20]:
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
print(rdf.shape)

(714, 13)


In [21]:
# 범주형 데이터의 결측치를 보강하는 방법 >> 최빈값으로 대체
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)

rdf['embarked'].isnull().sum()

0

In [22]:
ndf = rdf[['survived', 'pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [23]:
# one-hot encoding

onehot_sex = pd.get_dummies(ndf['sex'])  # male, female
ndf = pd.concat([ndf, onehot_sex], axis=1)
# onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')  # C, Q, S
onehot_embarked = pd.get_dummies(ndf['embarked'])  # C, Q, S
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,C,Q,S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [25]:
X = ndf[['pclass','age','sibsp','parch','female','male','C','Q','S']]
y = ndf['survived']

# Normalize variable X
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18)

print(X_train.shape, X_test.shape)

(499, 9) (215, 9)


In [27]:
# KNN Classification model
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)

print(y_hat[:10])
print(y_test.values[:10])

[1 0 0 1 1 0 1 1 1 0]
[1 0 0 1 1 0 1 1 0 0]


In [28]:
# model 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
print(knn_matrix)

[[102  14]
 [ 32  67]]


In [29]:
# model 성능 평가 - 평가 지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.76      0.88      0.82       116
           1       0.83      0.68      0.74        99

    accuracy                           0.79       215
   macro avg       0.79      0.78      0.78       215
weighted avg       0.79      0.79      0.78       215



# SVM

In [30]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
pd.set_option('display.max_columns', 15)

In [32]:
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)

In [33]:
ndf = rdf[['survived', 'pclass','sex','age','sibsp','parch','embarked']]

onehot_sex = pd.get_dummies(ndf['sex'])  # male, female
ndf = pd.concat([ndf, onehot_sex], axis=1)
onehot_embarked = pd.get_dummies(ndf['embarked'])  # C, Q, S
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex','embarked'], axis=1, inplace=True)

In [34]:
X = ndf[['pclass','age','sibsp','parch','female','male','C','Q','S']]
y = ndf['survived']

# Normalize variable X
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18)

print(X_train.shape, X_test.shape)

(499, 9) (215, 9)


In [36]:
from sklearn import svm

svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

y_hat = svm_model.predict(X_test)

In [37]:
from sklearn import metrics
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(svm_matrix)
print()

svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

[[103  13]
 [ 34  65]]

              precision    recall  f1-score   support

           0       0.75      0.89      0.81       116
           1       0.83      0.66      0.73        99

    accuracy                           0.78       215
   macro avg       0.79      0.77      0.77       215
weighted avg       0.79      0.78      0.78       215

